In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
!ls

MANIFEST.in                           gceline_frontend_streamlit.ipynb
Makefile                              notebooks
README.md                             raw_data
WorkforceSentimentMonitoring          requirements.txt
WorkforceSentimentMonitoring.egg-info scripts
app.py                                setup.py
build                                 test_data
dist                                  tests


In [2]:
cd ..

/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring


In [26]:
from WorkforceSentimentMonitoring.data import get_data, merge, holdout, drop_wrong_language, get_prepaired_data

In [18]:
from WorkforceSentimentMonitoring.feature_engineering import SCORE_COLS, FEATURE_COLS

In [142]:
from WorkforceSentimentMonitoring.encoders import Preprocessor, FeatureEngineer, CustomMinMaxScaler

# Load Data

In [19]:
# load a small dataset
submission, train, test = get_data()
df = merge(submission, train, test)

In [89]:
target = df[SCORE_COLS]
target

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,4,5,5,4,5,5
1,2,3,3,5,3,5
2,5,4,5,5,4,5
3,2,5,5,4,5,4
4,5,5,5,5,5,4
...,...,...,...,...,...,...
29267,4,4,5,4,4,4
29268,1,3,5,4,2,2
29269,3,3,3,3,3,3
29270,1,1,1,3,1,1


In [21]:
features = df[FEATURE_COLS]
features

,summary,positives,negatives,advice_to_mgmt,review
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,Best Company to work for People are smart and ...
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,"Moving at the speed of light, burn out is inev..."
2,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,Great balance between big-company security and...
3,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,The best place I've worked and also the most d...
4,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,Execellent for engineers Impact driven. Best t...
...,...,...,...,...,...
29267,great place to grow!,Great health benefits. Many internal job oppor...,Red Tape. Tough to negotiate internal politics...,NaN,great place to grow! Great health benefits. Ma...
29268,An ocean of opportunities,diverse set of people and problems they solve....,sometimes feels like a behemoth trying to get ...,align workgroups and incentives better to crea...,An ocean of opportunities diverse set of peopl...
29269,Tech Gaint,Equip its employees wid huge salaries :D,High qualification required to join it,Good Going....try searching new and bigger tal...,Tech Gaint Equip its employees wid huge salari...
29270,Terrible,They had great health benefits (no longer im t...,* 14 hour days * no idea how to run a service ...,Learn to excel (no pun intended) and ignore mo...,Terrible They had great health benefits (no lo...


In [90]:
small_df = df.sample(n=10_000, random_state=1, replace=True)

In [91]:
small_df = small_df.reset_index(drop=True)

In [92]:
small_df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,Terrible Company,Decent pay and health plan.,"Stressful work environment, very high quotas, ...",Stop treating your workers like trash.,1,1,1,2,1,1,Terrible Company Decent pay and health plan. S...
1,Amazon is great to buy and sell products in th...,its really cool place to work with amazon,itsw great job to work .join with us.,make easy to post products in facebook .,5,5,5,5,5,5,Amazon is great to buy and sell products in th...
2,Low paymasters,Good quality projects to work on.,Unrealistic deadlines and if you have a bad bo...,Dont bring in people at the wrong pay grade/le...,2,3,3,1,3,3,Low paymasters Good quality projects to work o...
3,Buying,"Startup Feel, Fun Environment, Great Company","Long hours, Poor Management, Growth",NaN,2,3,3,3,2,2,"Buying Startup Feel, Fun Environment, Great Co..."
4,Ok,"Smart people, good compensation, the opportuni...",A lack of good communication between the diffe...,NaN,3,3,4,5,3,3,"Ok Smart people, good compensation, the opport..."


In [93]:
small_target = small_df[SCORE_COLS]
small_feature = small_df[FEATURE_COLS]

In [110]:
small_target

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,1,1,1,2,1,1
1,5,5,5,5,5,5
2,2,3,3,1,3,3
3,2,3,3,3,2,2
4,3,3,4,5,3,3
...,...,...,...,...,...,...
9995,3,5,4,4,5,3
9996,1,1,1,2,3,3
9997,5,5,4,5,4,4
9998,3,3,3,5,2,2


## Group in 2 classes: Test in a small Dataset

In [109]:
for key, values in small_target.items():
    print(f"key: {key}")
    #print(f"values: {values}")

key: work-balance
key: culture-values
key: career-opportunities
key: comp-benefits
key: senior-mgmt
key: overall


In [129]:
# remap scores into 2 class targets (leave score 3 as '3')
def remap(data, dict_labels):
    for key, values in dict_labels.items():
        data.replace({key: {1: '0', 2: '0', 3: '1', 4: '2', 5: '2'}}, inplace=True)
    return data

In [114]:
small_target.apply(lambda row: remap(small_target, small_target), axis=1)

I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping c

I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping 

I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balanc

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balanc

...

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapp

I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-valu

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

0            work-balance culture-values career-opport...
1            work-balance culture-values career-opport...
2            work-balance culture-values career-opport...
3            work-balance culture-values career-opport...
4            work-balance culture-values career-opport...
                              ...                        
9995         work-balance culture-values career-opport...
9996         work-balance culture-values career-opport...
9997         work-balance culture-values career-opport...
9998         work-balance culture-values career-opport...
9999         work-balance culture-values career-opport...
Length: 10000, dtype: object

In [125]:
small_target = small_target.astype('uint8')

In [126]:
small_target.dtypes

work-balance            uint8
culture-values          uint8
career-opportunities    uint8
comp-benefits           uint8
senior-mgmt             uint8
overall                 uint8
dtype: object

In [127]:
small_target

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,0,0,0,0,0,0
1,1,1,1,1,1,1
2,0,3,3,0,3,3
3,0,3,3,3,0,0
4,3,3,1,1,3,3
...,...,...,...,...,...,...
9995,3,1,1,1,1,3
9996,0,0,0,0,3,3
9997,1,1,1,1,1,1
9998,3,3,3,1,0,0


In [42]:
import pandas as pd

In [118]:
'''small_target['work-balance'] = small_target['work-balance'].apply(lambda x: {1: "0", 2: "0", 3: '3', 4: '1', 5: '1'}.get(x, x))
pd.DataFrame(small_target['work-balance'])'''

'small_target[\'work-balance\'] = small_target[\'work-balance\'].apply(lambda x: {1: "0", 2: "0", 3: \'3\', 4: \'1\', 5: \'1\'}.get(x, x))\npd.DataFrame(small_target[\'work-balance\'])'

In [117]:
'''def remap(df):
    for _ in df.columns:
        df["_"] = df["_"].apply(lambda x: {1: "0", 2: "0", 3: '3', 4: '1', 5: '1'}.get(x,x))
        pd.DataFrame(df["_"])
    return df'''

'def remap(df):\n    for _ in df.columns:\n        df["_"] = df["_"].apply(lambda x: {1: "0", 2: "0", 3: \'3\', 4: \'1\', 5: \'1\'}.get(x,x))\n        pd.DataFrame(df["_"])\n    return df'

In [116]:
'''for col in small_target.columns:
    small_target.col = small_target.col.apply(lambda x: {1: '0', 2: '0', 3: '3', 4: '1', 5: '1'}.get(x,x))
    pd.DataFrame(small_target.col)'''

"for col in small_target.columns:\n    small_target.col = small_target.col.apply(lambda x: {1: '0', 2: '0', 3: '3', 4: '1', 5: '1'}.get(x,x))\n    pd.DataFrame(small_target.col)"

In [115]:
#X_train, X_test, y_train, y_test = get_prepaired_data()

## holdout

In [121]:
X_train, X_test, y_train, y_test = holdout(small_df, small_target)

ValueError: Boolean array expected for the condition, not object

In [7]:
X_train.head()

,summary,positives,negatives,advice_to_mgmt,review
0,Fulfullment Associate,Good about closing for inclement weather,Work as a picker is very physical,NaN,Fulfullment Associate Good about closing for i...
1,"great company, bad orgs",never have to worry about benefits,internal politics are increasing which harm IC,Please fix the review process. It is completel...,"great company, bad orgs never have to worry ab..."
2,"Dynamic, fast-paced, but poorly managed.",Apple prepares their employees for success wit...,The way Apple cultivates a management team is ...,Realize the history between managers on a stor...,"Dynamic, fast-paced, but poorly managed. Apple..."
3,Apple,Great work environment. Excellent benefits. Pe...,Not much opportunity to advance in certain are...,NaN,Apple Great work environment. Excellent benefi...
4,SDE2,"Best in class, you name the benefits and it’s ...","No as such, keep it up",Keep it up,"SDE2 Best in class, you name the benefits and ..."


In [120]:
# drop non-eng reviews
# small_df = drop_wrong_language(small_df, 'review', language = 'en', inplace=False)

In [11]:
y_train.head()

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,3,4,3,2,3,3
1,4,3,3,3,3,4
2,4,1,3,5,1,3
3,4,4,3,5,3,3
4,5,5,5,5,3,4


# Prepare 2-class target

In [130]:
target

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,4,5,5,4,5,5
1,2,3,3,5,3,5
2,5,4,5,5,4,5
3,2,5,5,4,5,4
4,5,5,5,5,5,4
...,...,...,...,...,...,...
29267,4,4,5,4,4,4
29268,1,3,5,4,2,2
29269,3,3,3,3,3,3
29270,1,1,1,3,1,1


In [131]:
# transform scores into 2 classes: 0 => negative; 1 => positive
target.apply(lambda row: remap(target, target), axis=1)

0              work-balance culture-values career-oppor...
1              work-balance culture-values career-oppor...
2              work-balance culture-values career-oppor...
3              work-balance culture-values career-oppor...
4              work-balance culture-values career-oppor...
                               ...                        
29267          work-balance culture-values career-oppor...
29268          work-balance culture-values career-oppor...
29269          work-balance culture-values career-oppor...
29270          work-balance culture-values career-oppor...
29271          work-balance culture-values career-oppor...
Length: 52815, dtype: object

In [132]:
target

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,2,2,2,2,2,2
1,0,1,1,2,1,2
2,2,2,2,2,2,2
3,0,2,2,2,2,2
4,2,2,2,2,2,2
...,...,...,...,...,...,...
29267,2,2,2,2,2,2
29268,0,1,2,2,0,0
29269,1,1,1,1,1,1
29270,0,0,0,1,0,0


In [134]:
df_copy = df.copy()

In [135]:
df_copy

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,4,5,5,4,5,5,Best Company to work for People are smart and ...
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,2,3,3,5,3,5,"Moving at the speed of light, burn out is inev..."
2,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5,4,5,5,4,5,Great balance between big-company security and...
3,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,2,5,5,4,5,4,The best place I've worked and also the most d...
4,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,5,5,5,5,5,4,Execellent for engineers Impact driven. Best t...
...,...,...,...,...,...,...,...,...,...,...,...
29267,great place to grow!,Great health benefits. Many internal job oppor...,Red Tape. Tough to negotiate internal politics...,NaN,4,4,5,4,4,4,great place to grow! Great health benefits. Ma...
29268,An ocean of opportunities,diverse set of people and problems they solve....,sometimes feels like a behemoth trying to get ...,align workgroups and incentives better to crea...,1,3,5,4,2,2,An ocean of opportunities diverse set of peopl...
29269,Tech Gaint,Equip its employees wid huge salaries :D,High qualification required to join it,Good Going....try searching new and bigger tal...,3,3,3,3,3,3,Tech Gaint Equip its employees wid huge salari...
29270,Terrible,They had great health benefits (no longer im t...,* 14 hour days * no idea how to run a service ...,Learn to excel (no pun intended) and ignore mo...,1,1,1,3,1,1,Terrible They had great health benefits (no lo...


In [138]:
df_copy[SCORE_COLS] = target

In [139]:
df_copy

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,2,2,2,2,2,2,Best Company to work for People are smart and ...
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,0,1,1,2,1,2,"Moving at the speed of light, burn out is inev..."
2,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,2,2,2,2,2,2,Great balance between big-company security and...
3,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,0,2,2,2,2,2,The best place I've worked and also the most d...
4,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,2,2,2,2,2,2,Execellent for engineers Impact driven. Best t...
...,...,...,...,...,...,...,...,...,...,...,...
29267,great place to grow!,Great health benefits. Many internal job oppor...,Red Tape. Tough to negotiate internal politics...,NaN,2,2,2,2,2,2,great place to grow! Great health benefits. Ma...
29268,An ocean of opportunities,diverse set of people and problems they solve....,sometimes feels like a behemoth trying to get ...,align workgroups and incentives better to crea...,0,1,2,2,0,0,An ocean of opportunities diverse set of peopl...
29269,Tech Gaint,Equip its employees wid huge salaries :D,High qualification required to join it,Good Going....try searching new and bigger tal...,1,1,1,1,1,1,Tech Gaint Equip its employees wid huge salari...
29270,Terrible,They had great health benefits (no longer im t...,* 14 hour days * no idea how to run a service ...,Learn to excel (no pun intended) and ignore mo...,0,0,0,1,0,0,Terrible They had great health benefits (no lo...


## Prepare Features

In [141]:
features

,summary,positives,negatives,advice_to_mgmt,review
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,Best Company to work for People are smart and ...
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,"Moving at the speed of light, burn out is inev..."
2,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,Great balance between big-company security and...
3,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,The best place I've worked and also the most d...
4,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,Execellent for engineers Impact driven. Best t...
...,...,...,...,...,...
29267,great place to grow!,Great health benefits. Many internal job oppor...,Red Tape. Tough to negotiate internal politics...,NaN,great place to grow! Great health benefits. Ma...
29268,An ocean of opportunities,diverse set of people and problems they solve....,sometimes feels like a behemoth trying to get ...,align workgroups and incentives better to crea...,An ocean of opportunities diverse set of peopl...
29269,Tech Gaint,Equip its employees wid huge salaries :D,High qualification required to join it,Good Going....try searching new and bigger tal...,Tech Gaint Equip its employees wid huge salari...
29270,Terrible,They had great health benefits (no longer im t...,* 14 hour days * no idea how to run a service ...,Learn to excel (no pun intended) and ignore mo...,Terrible They had great health benefits (no lo...


In [143]:
from WorkforceSentimentMonitoring.feature_engineering import get_lengths,\
get_linear_regression_cols, get_subjectivity_polarity_columns,\
getPolarity, getSubjectivity

In [144]:

pipe_preprocessor = Preprocessor(features)
#pipe_scaler = Pipeline(features, CustomMinMaxScaler)


TypeError: __init__() takes 1 positional argument but 2 were given